In [1]:
import requests
import csv
import pandas as pd
import io
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.graphics as sgr
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sc
from matplotlib.collections import LineCollection
from statsmodels.stats.outliers_influence import summary_table

 QUESTION 1

Function getdataframes returns a dataframe containing all firms plus the SP500TR index. We keep the 'Adj Close' column
as it appears in finance.yahoo.com, and combine each dataframe a on common 'Date' axis. 

In [2]:
def getdataframes():
    list_ =[]
    firms = {"NDAQ","^SP500TR","BK","BFED","BLK","CARV","C","CME","EVR","FII","FIG","GBL","LAZ","NTRS","BPOP","PFS","STT","BX","BX"}
    frame = pd.DataFrame()
    for x in firms:
        request = 'http://chart.finance.yahoo.com/table.csv?s={}&a=10&b=21&c=2008&d=11&e=10&f=2008&g=d&ignore=.csv'.format(str(x))
        y = pd.read_csv(io.StringIO(requests.get(request).content.decode('utf-8')),usecols=['Adj Close','Date'],index_col=0)
        y.rename(columns={"Adj Close" : str(x)},inplace=True)
        list_.append(y)
    frame=pd.concat(list_, axis=1,join='outer')
    frame = frame.rename(columns={"^SP500TR": "SP500"})
    return frame

In [3]:
returns = getdataframes()
returns

,BX,EVR,C,STT,LAZ,FII,CARV,SP500,NTRS,NDAQ,BPOP,CME,BK,PFS,GBL,BLK,BFED,FIG
2008-11-21,2.823453,5.783971,37.017329,28.267192,17.594389,11.565914,87.436466,1282.589966,31.701180,15.678543,54.932721,25.959218,22.217093,10.028812,11.231554,88.556553,7.5340,1.479990
2008-11-24,3.904259,6.255800,58.422574,32.100785,19.956206,12.432865,87.301947,1365.630005,36.672768,17.568534,57.793801,29.640884,27.728376,11.319946,11.945425,94.435924,7.4594,1.731445
2008-11-25,3.910628,6.949971,59.699033,33.577296,20.148537,12.051932,81.290548,1374.790039,34.725913,17.801292,58.079912,29.527171,25.578890,11.079736,11.590653,95.493075,NaN,1.724260
2008-11-26,4.006165,8.536647,69.223386,36.156745,23.087346,12.485407,76.508751,1423.900024,37.107260,20.073003,58.652126,31.272752,25.759446,11.229867,11.850242,101.429375,7.6834,2.169694
2008-11-28,3.987058,8.272201,81.398849,37.455363,24.048998,13.037103,76.508751,1437.680054,38.343890,20.017142,59.605819,31.300810,25.974395,11.252387,11.867548,102.226299,8.1221,2.191248
2008-12-01,3.541220,8.230882,63.332034,31.513738,19.571545,11.670999,76.508751,1309.390015,32.921101,17.503361,51.117949,25.495503,22.225691,9.458311,9.881692,87.603840,7.8421,1.796105
2008-12-02,3.401100,8.958108,70.892600,32.847935,19.533079,12.787526,76.508751,1361.790039,35.611605,18.639218,56.363261,25.489597,23.334825,10.208971,10.106669,92.758935,7.5433,1.796105
2008-12-03,3.846937,8.958108,76.783952,32.554410,20.140843,13.464011,76.508751,1397.719971,38.059799,19.039561,58.366019,26.920617,25.441323,10.501727,11.270493,99.665125,7.2352,1.343486
2008-12-04,3.649495,9.346513,72.660008,33.194826,19.309976,12.767823,76.508751,1356.810059,38.577846,20.240589,55.981786,26.207322,24.538539,10.126398,10.976292,94.706413,7.8421,1.357855
2008-12-05,3.993427,9.916725,75.703872,35.827641,19.640785,14.035411,76.508751,1406.359985,42.262673,21.739547,55.791047,28.734130,25.897015,10.689392,12.025923,104.992049,7.8421,1.264458


Insert 'cumulative' column : sum of all firms minus the SP500 

In [4]:
returns['cumulative'] = returns.sum(axis=1) - returns['SP500']
returns.sample()

,BX,EVR,C,STT,LAZ,FII,CARV,SP500,NTRS,NDAQ,BPOP,CME,BK,PFS,GBL,BLK,BFED,FIG,cumulative
2008-12-02,3.4011,8.958108,70.8926,32.847935,19.533079,12.787526,76.508751,1361.790039,35.611605,18.639218,56.363261,25.489597,23.334825,10.208971,10.106669,92.758935,7.5433,1.796105,506.781585


In [5]:
mod = smf.ols("cumulative ~ SP500", data=returns)
rels = mod.fit()
rels.params

Intercept   -367.471325
SP500          0.645168
dtype: float64

In [6]:
rels.summary()

C:\Users\chrys\Anaconda3\lib\site-packages\scipy\stats\stats.py:1327: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             cumulative   R-squared:                       0.938
Model:                            OLS   Adj. R-squared:                  0.932
Method:                 Least Squares   F-statistic:                     166.0
Date:                Wed, 30 Nov 2016   Prob (F-statistic):           5.58e-08
Time:                        23:54:28   Log-Likelihood:                -46.205
No. Observations:                  13   AIC:                             96.41
Df Residuals:                      11   BIC:                             97.54
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   -367.4713     69.564     -5.282      0.000      -520.581  -214.362
SP500          0.6452      0.050     12.885      0.000         0.535     0.755
==============================================================================
Omnibus:                        0.572   Durbin-Watson:                   1.168
Prob(Omnibus):                  0.751   Jarque-Bera (JB):                0.568
Skew:                          -0.387   Prob(JB):                        0.753
Kurtosis:                       2.328   Cond. No.                     3.79e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.79e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [7]:
rels.resid

2008-11-21    -0.206838
2008-11-24     3.080912
2008-11-25   -16.270945
2008-11-26   -13.952391
2008-11-28    -4.464482
2008-12-01     2.909060
2008-12-02    -4.331017
2008-12-03    -4.193098
2008-12-04    10.092856
2008-12-05     6.997222
2008-12-08    -1.425386
2008-12-09    12.744898
2008-12-10     9.019208
dtype: float64

Now try an R^2 test and concatenate the residuals to the returns table

In [8]:
mod=sm.OLS(returns['cumulative'],returns['SP500'],data=returns)
res=mod.fit()
res.summary()
res.resid

C:\Users\chrys\Anaconda3\lib\site-packages\scipy\stats\stats.py:1327: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=13
  "anyway, n=%i" % int(n))


2008-11-21   -28.648313
2008-11-24    -3.410405
2008-11-25   -20.340969
2008-11-26    -5.041064
2008-11-28     8.089350
2008-12-01   -18.448297
2008-12-02   -11.837361
2008-12-03    -2.202002
2008-12-04     1.270143
2008-12-05    11.272152
2008-12-08    17.186920
2008-12-09    22.443930
2008-12-10    23.243606
dtype: float64

Run t-test and get the p-values (use scipy.stats library) to determine significant statistical difference between cumulative and SP500

In [9]:
sc.ttest_1samp(returns['cumulative'],returns['SP500'])[1]

array([  1.53193733e-17,   4.38441647e-18,   3.84857246e-18,
         1.95808310e-18,   1.63066915e-18,   1.00840409e-17,
         4.63263390e-18,   2.79396077e-18,   4.97746874e-18,
         2.48177114e-18,   1.21008646e-18,   1.88167072e-18,
         1.50079545e-18])

In [10]:
returns['stat_difference'] = sc.ttest_1samp(returns['cumulative'],returns['SP500'])[1]

In [11]:
returns

,BX,EVR,C,STT,LAZ,FII,CARV,SP500,NTRS,NDAQ,BPOP,CME,BK,PFS,GBL,BLK,BFED,FIG,cumulative,stat_difference
2008-11-21,2.823453,5.783971,37.017329,28.267192,17.594389,11.565914,87.436466,1282.589966,31.701180,15.678543,54.932721,25.959218,22.217093,10.028812,11.231554,88.556553,7.5340,1.479990,459.808378,1.531937e-17
2008-11-24,3.904259,6.255800,58.422574,32.100785,19.956206,12.432865,87.301947,1365.630005,36.672768,17.568534,57.793801,29.640884,27.728376,11.319946,11.945425,94.435924,7.4594,1.731445,516.670939,4.384416e-18
2008-11-25,3.910628,6.949971,59.699033,33.577296,20.148537,12.051932,81.290548,1374.790039,34.725913,17.801292,58.079912,29.527171,25.578890,11.079736,11.590653,95.493075,NaN,1.724260,503.228847,3.848572e-18
2008-11-26,4.006165,8.536647,69.223386,36.156745,23.087346,12.485407,76.508751,1423.900024,37.107260,20.073003,58.652126,31.272752,25.759446,11.229867,11.850242,101.429375,7.6834,2.169694,537.231612,1.958083e-18
2008-11-28,3.987058,8.272201,81.398849,37.455363,24.048998,13.037103,76.508751,1437.680054,38.343890,20.017142,59.605819,31.300810,25.974395,11.252387,11.867548,102.226299,8.1221,2.191248,555.609961,1.630669e-18
2008-12-01,3.541220,8.230882,63.332034,31.513738,19.571545,11.670999,76.508751,1309.390015,32.921101,17.503361,51.117949,25.495503,22.225691,9.458311,9.881692,87.603840,7.8421,1.796105,480.214822,1.008404e-17
2008-12-02,3.401100,8.958108,70.892600,32.847935,19.533079,12.787526,76.508751,1361.790039,35.611605,18.639218,56.363261,25.489597,23.334825,10.208971,10.106669,92.758935,7.5433,1.796105,506.781585,4.632634e-18
2008-12-03,3.846937,8.958108,76.783952,32.554410,20.140843,13.464011,76.508751,1397.719971,38.059799,19.039561,58.366019,26.920617,25.441323,10.501727,11.270493,99.665125,7.2352,1.343486,530.100362,2.793961e-18
2008-12-04,3.649495,9.346513,72.660008,33.194826,19.309976,12.767823,76.508751,1356.810059,38.577846,20.240589,55.981786,26.207322,24.538539,10.126398,10.976292,94.706413,7.8421,1.357855,517.992532,4.977469e-18
2008-12-05,3.993427,9.916725,75.703872,35.827641,19.640785,14.035411,76.508751,1406.359985,42.262673,21.739547,55.791047,28.734130,25.897015,10.689392,12.025923,104.992049,7.8421,1.264458,546.864946,2.481771e-18


So there is high statistical difference 

QUESTION 2

Get prior days

In [12]:
def getpriordates():
    list_ =[]
    firms = {"NDAQ","^SP500TR","BK","BFED","BLK","CARV","C","CME","EVR","FII","FIG","GBL","LAZ","NTRS","BPOP","PFS","STT","BX","BX"}
    newframe = pd.DataFrame()
    for z in firms:
        request = 'http://chart.finance.yahoo.com/table.csv?s={}&a=10&b=7&c=2007&d=9&e=22&f=2008&g=d&ignore=.csv'.format(str(z))
        w = pd.read_csv(io.StringIO(requests.get(request).content.decode('utf-8')),usecols=['Adj Close','Date'],index_col=0)
        w.rename(columns={"Adj Close" : str(z)},inplace=True)
        list_.append(w)
    newframe=pd.concat(list_, axis=1,join='outer')
    newframe = newframe.rename(columns={"^SP500TR": "SP500"})
    return newframe

In [13]:
priorreturns = getpriordates()
priorreturns.head(20)

,BX,EVR,C,STT,LAZ,FII,CARV,SP500,NTRS,NDAQ,BPOP,CME,BK,PFS,GBL,BLK,BFED,FIG
Date,,,,,,,,,,,,,,,,,,
2008-10-22,5.404029,10.290750,129.167093,32.074102,23.887217,13.936205,90.395857,1433.219971,42.722231,28.591926,57.412326,44.919875,24.254805,9.301747,13.130881,97.477264,7.8701,2.801923
2008-10-21,5.464748,10.388371,137.506716,35.222805,26.670864,14.886104,84.611594,1526.020020,48.621289,30.807778,62.753009,48.540992,26.963159,9.204854,13.723610,105.877527,7.8701,3.383861
2008-10-20,6.290532,9.940947,146.331186,35.578592,27.606415,14.892611,84.073525,1574.500000,48.805113,31.133637,65.327978,53.212119,26.835170,9.309200,14.255768,116.465284,7.6087,3.814926
2008-10-17,6.606273,9.778247,144.294777,36.254584,27.222993,14.580315,84.073525,1502.839966,47.025367,28.852615,61.990052,53.932796,25.435815,8.943988,14.039444,120.937837,7.7394,3.987352
2008-10-16,5.683338,10.502260,154.185948,40.105968,27.989838,14.690920,84.073525,1512.209961,48.679778,26.152629,64.469653,55.134912,26.519461,9.614786,13.974545,119.026832,7.6927,3.951430
2008-10-15,5.586187,9.306418,157.386036,41.653638,26.877912,14.482723,84.073525,1450.500000,45.897361,24.905049,66.186303,53.142708,24.957987,9.756400,14.710049,116.245719,7.6553,3.786189
2008-10-14,6.600201,10.762579,180.562409,50.423758,29.715239,17.078682,87.301947,1594.410034,53.283720,28.228825,71.813094,58.835777,29.659473,10.322853,16.765128,137.608262,7.8047,5.402683
2008-10-13,5.871568,10.168726,152.731363,43.005622,28.373260,17.729299,80.710584,1602.930054,52.648692,27.958827,71.336247,60.761528,26.178155,10.076893,17.111248,146.244352,7.9354,4.166963
2008-10-10,4.851482,10.119916,136.827913,38.424880,28.143207,15.875042,70.621761,1436.560059,47.267681,24.085743,70.954769,58.056026,22.611510,9.264481,16.596397,129.297444,7.8981,2.873767


Create Schedule and Personal Connections dataframes

In [14]:
schedule = pd.DataFrame({'Firm' : ['BK','BFED','BLK','CME','EVR','FII','LAZ','NTRS','PFS','STT','BX','NASQ'], 'Schedule_Connections' :[7,1,13,2,1,1,1,1,2,1,6,2] })
personal = pd.DataFrame({'Firm' : ['BLK','CARV','C','FIG','GBL','BPOR','BX','NASQ'], 'Personal_Connections':[2,1,2,1,1,1,4,1]})
schedule
personal


,Firm,Personal_Connections
0,BLK,2
1,CARV,1
2,C,2
3,FIG,1
4,GBL,1
5,BPOR,1
6,BX,4
7,NASQ,1


In [15]:
personal = pd.DataFrame({'Firm' : ['BLK','CARV','C','FIG','GBL','BPOR','BX','NASQ'], 'Personal_Connections':[2,1,2,1,1,1,4,1]})

Regression to estimate a and b of Return equation

In [16]:
firms = {"NDAQ","BK","BFED","BLK","CARV","C","CME","EVR","FII","FIG","GBL","LAZ","NTRS","BPOP","PFS","STT","BX","BX"}
for x in firms:
    mod = smf.ols("{} ~ SP500".format(str(x)), data=priorreturns)
    rels = mod.fit()
    print(rels.summary())

                            OLS Regression Results                            
Dep. Variable:                     BX   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.756
Method:                 Least Squares   F-statistic:                     748.6
Date:                Wed, 30 Nov 2016   Prob (F-statistic):           9.85e-76
Time:                        23:54:32   Log-Likelihood:                -284.04
No. Observations:                 242   AIC:                             572.1
Df Residuals:                     240   BIC:                             579.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -5.5607      0.592     -9.387      0.0